# Setup

In [1]:
# Clone repo
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor.git
  
# Change directory
%cd /content/keras_triplet_descriptor  

Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip

  
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

import csv

def pushCSV(log, title, metric='acc'):
 
  with open(title, mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['epoch','loss','val_loss', metric,'val_' + metric])
    for i in range(len(log)):
      
      his = log[i].history
      writer.writerow([i+1,his['loss'], his['val_loss'], his[metric], his['val_' + metric]])
  
  
  from google.colab import files
  files.download(title)

random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs)) 

Cloning into 'keras_triplet_descriptor'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 181 (delta 0), reused 1 (delta 0), pack-reused 178
Receiving objects: 100% (181/181), 149.87 MiB | 20.22 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (69/69), done.
/content/keras_triplet_descriptor


Using TensorFlow backend.


Declare models

In [0]:
def get_denoise_model(shape):
  
  
  init_weights = keras.initializers.glorot_normal()
    
  inputs = Input(shape)
 
  conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(pool1)
  pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
  conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(pool2)
  pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
  conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(pool3)
  drop4 = Dropout(0.5)(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

  conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(pool4)
  drop5 = Dropout(0.5)(conv5)

  ## Now the decoder starts

  up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(UpSampling2D(size = (2,2))(drop5))
  merge6 = concatenate([drop4,up6], axis = 3)
  conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(merge6)

  up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(UpSampling2D(size = (2,2))(conv6))
  merge7 = concatenate([conv3,up7], axis = 3)
  conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(merge7)

  up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(UpSampling2D(size = (2,2))(conv7))
  merge8 = concatenate([conv2,up8], axis = 3)
  conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(merge8)

  up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(UpSampling2D(size = (2,2))(conv8))
  merge9 = concatenate([conv1,up9], axis = 3)
  conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = init_weights)(merge9)
  conv10 = Conv2D(1, 3,  padding = 'same')(conv9)

  model = Model(inputs = inputs, outputs = conv10)
  
  return model
  




def get_baseline_DN_model(shape):
  
  inputs = Input(shape)
  
  ## Encoder starts 
  conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
  pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
  
  ## Bottleneck
  conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)

  ## Now the decoder starts
  up3 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv2))
  merge3 = concatenate([conv1,up3], axis = -1)
  conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge3)
    
  conv4 = Conv2D(1, 3,  padding = 'same')(conv3)

  shallow_net = Model(inputs = inputs, outputs = conv4)
  
  return shallow_net




def get_BL_descriptor_model(shape):
  
  '''Architecture copies HardNet architecture'''
  
  init_weights = keras.initializers.glorot_normal()
  reg = keras.regularizers.l1(0.01)
  
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  
  # Final descriptor reshape
  descriptor_model.add(Reshape((128,)))
  
  return descriptor_model


def get_descriptor_model(shape):
  model = keras.applications.xception.Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(75, 75, 3), pooling=None)
  
  newInput = Input(batch_shape=(None, 32, 32, 1))
  resizedImg = Lambda(lambda image: tf.image.resize_images(image, (75, 75)))(newInput)
  resizedRGB = concatenate([resizedImg,resizedImg,resizedImg])
  newOutputs = model(resizedRGB)
  model = Model(newInput, newOutputs)

  # Freeze all the layers
  for layer in model.layers[:]:
      layer.trainable = False

  # Add Dense layer to classify on CIFAR100
  output = model.output
  output = Dense(units=128, activation='softmax')(output)
  model = Model(model.input, output)
  
  return model
  
def triplet_loss(x):
  
  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

#Train Baseline models

Denoiser

In [0]:
denoise_generator = DenoiseHPatches(random.sample(seqs_train, 3), batch_size=50)   # Takes 3 sample sequences and outputs clean as labe
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)    # Takes 1 sample sequence and outputs clean as label

shape = (32, 32, 1)
denoise_model = get_baseline_DN_model(shape)


adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
denoise_log = []
denoise_model.compile(loss='mean_absolute_error', optimizer=adam, metrics=['acc'])

epochs = 30
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  print("\n/////          " + str(e) + "      ////////    ")
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val)
  denoise_log.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise.h5" https://file.io
    
del denoise_generator
del denoise_generator_val

Descriptor

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_BL_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

desc_log = []

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
# sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=adam, metrics=['acc'])

### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)
plot_triplet(training_generator)

epochs = 30
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  print("\n/////          " + str(e) + "      ////////    ")
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val)
  denoise_log.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise.h5" https://file.io


del hPatches
del training_generator
del val_generator

Run benchmark

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

Import what's necessary
Fix seeds
Import and split data


# Improved models

Setup

In [0]:
!cd ..
!rm -r /content/keras_triplet_descriptor

# Clone repo
!git clone https://github.com/ms6616/DL_CW_HPatches.git
  
# Change directory
%cd /content/DL_CW_HPatches  

# Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip

  
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

Denoiser

In [0]:
denoise_generator = DenoiseHPatches(random.sample(seqs_train, 6), batch_size=50)   # Takes 6 sample sequences and outputs clean as labe
denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 1), batch_size=50)    # Takes 1 sample sequence and outputs clean as label

shape = (32, 32, 1)
denoise_model = get_denoise_model(shape)


adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
denoise_log = []
denoise_model.compile(loss='mean_absolute_error', optimizer=adam, metrics=['acc'])

epochs = 30
### Use a loop to save for each epoch the weights in an external website in
### case colab stops. Every time you call fit/fit_generator the weigths are NOT
### reset, so e.g. calling 5 times fit(epochs=1) behave as fit(epochs=5)
for e in range(epochs):
  print("\n/////          " + str(e) + "      ////////    ")
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, 
                                                epochs=1, verbose=1, 
                                                validation_data=denoise_generator_val)
  denoise_log.append(denoise_history)
  ### Saves optimizer and weights
  denoise_model.save('denoise.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@denoise.h5" https://file.io
    
del denoise_generator
del denoise_generator_val

Train Descriptor

In [0]:
from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_BL_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

desc_log = []

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
# sgd = keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=adam, metrics=['acc'])

### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,
                    denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)
plot_triplet(training_generator)

epochs = 50

for e in range(epochs):
  print("\n/////          " + str(e) + "      ////////    ")
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  
  desc_log.append(descriptor_history)
  
  ### Saves optimizer and weights
  descriptor_model_trip.save('descriptor.h5') 
  ### Uploads files to external hosting
  !curl -F "file=@descriptor.h5" https://file.io

Run benchmark

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model=denoise_model, use_clean=False)

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/DL_CW_HPatches/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval